In [1]:
from rtlsdr import RtlSdr
import  adi
import numpy as np  
import scipy.signal as signal

import matplotlib  
matplotlib.use('Agg') # necessary for headless mode  
# see http://stackoverflow.com/a/3054314/3524528
import matplotlib.pyplot as plt

In [2]:
sdr = adi.Pluto("ip:192.168.2.1")

F_station = int(101.8e6)  
Fs = int(1024000)         # Sample rate  
N = int(2097152)  # Samples to capture = 2^21

# configure device
sdr.sample_rate = Fs      # Hz  
sdr.rx_lo = F_station      # Hz  
sdr.rx_buffer_size = N
sdr.gain_control_mode_chan0 = 'manual'

# Read samples
samples = sdr.rx()

# Clean up the SDR device
# sdr.rx_destroy_buffer()  
# del(sdr)

# Convert samples to a numpy array
x1 = np.array(samples).astype("complex64")

In [7]:
plt.specgram(x1, NFFT=2048, Fs=Fs)  
plt.title("x1")  
plt.ylim(-Fs/2, Fs/2)  
plt.savefig("x1_spec.pdf", bbox_inches='tight', pad_inches=0.5)  
plt.close()

In [8]:
# To mix the data down, generate a digital complex exponential 
# (with the same length as x1) with phase -F_offset/Fs

# fc1 = np.exp(-1.0j*2.0*np.pi* F_offset/Fs*np.arange(len(x1)))  

# Now, just multiply x1 and the digital complex expontential
x2 = x1


In [9]:
plt.specgram(x2, NFFT=2048, Fs=Fs)  
plt.title("x2")  
plt.xlabel("Time (s)")  
plt.ylabel("Frequency (Hz)")  
plt.ylim(-Fs/2, Fs/2)  
plt.xlim(0,len(x2)/Fs)  
plt.ticklabel_format(style='plain', axis='y' ) 
plt.savefig("x2_spec.pdf", bbox_inches='tight', pad_inches=0.5)  
plt.close()  

In [10]:
# An FM broadcast signal has  a bandwidth of 200 kHz
f_bw = 200000  
dec_rate = int(Fs / f_bw)  
x4 = signal.decimate(x2, dec_rate)  
# Calculate the new sampling rate
Fs_y = Fs/dec_rate  

In [11]:
plt.specgram(x4, NFFT=2048, Fs=Fs_y)  
plt.title("x4")  
plt.ylim(-Fs_y/2, Fs_y/2)  
plt.xlim(0,len(x4)/Fs_y)  
plt.ticklabel_format(style='plain', axis='y' )  
plt.savefig("x4_spec.pdf", bbox_inches='tight', pad_inches=0.5)  
plt.show()
plt.close() 

/tmp/ipykernel_37463/1056615786.py:7: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [12]:
# Plot the constellation of x4.  What does it look like?
plt.scatter(np.real(x4[0:50000]), np.imag(x4[0:50000]), color="red", alpha=0.05)  
plt.title("x4")  
plt.xlabel("Real")  
plt.xlim(-1.1,1.1)  
plt.ylabel("Imag")  
plt.ylim(-1.1,1.1)  
plt.savefig("x4_const.pdf", bbox_inches='tight', pad_inches=0.5)  
plt.close() 

In [13]:
y5 = x4[1:] * np.conj(x4[:-1])  
x5 = np.angle(y5)  

In [14]:
# Note: x5 is now an array of real, not complex, values
# As a result, the PSDs will now be plotted single-sided by default (since
# a real signal has a symmetric spectrum)
# Plot the PSD of x5
plt.psd(x5, NFFT=2048, Fs=Fs_y, color="blue")  
plt.title("x5")  
plt.axvspan(0,             15000,         color="red", alpha=0.2)  
plt.axvspan(19000-500,     19000+500,     color="green", alpha=0.4)  
plt.axvspan(19000*2-15000, 19000*2+15000, color="orange", alpha=0.2)  
plt.axvspan(19000*3-1500,  19000*3+1500,  color="blue", alpha=0.2)  
plt.ticklabel_format(style='plain', axis='y' )  
plt.savefig("x5_psd.pdf", bbox_inches='tight', pad_inches=0.5)  
plt.close()  

In [15]:
# The de-emphasis filter
# Given a signal 'x5' (in a numpy array) with sampling rate Fs_y
# d = Fs_y * 75e-6   # Calculate the # of samples to hit the -3dB point  
# x = np.exp(-1/d)   # Calculate the decay between each sample  
# b = [1-x]          # Create the filter coefficients  
# print(b)
# a = [1,-x]  
# print(a)
# x6 = signal.lfilter(b,a,x5)
x6=x5

In [16]:
# Find a decimation rate to achieve audio sampling rate between 44-48 kHz
audio_freq = 44100.0  
dec_audio = int(Fs_y/audio_freq)  
Fs_audio = Fs_y / dec_audio

x7 = signal.decimate(x6, dec_audio) 

In [17]:
# Scale audio to adjust volume
x7 *= 10000 / np.max(np.abs(x7))  
# Save to file as 16-bit signed single-channel audio samples
x7.astype("int16").tofile("wbfm-mono.raw")  

In [18]:
print(Fs_audio)  

51200.0


In [20]:
!aplay wbfm-mono.raw -r 45600 -f S16_LE -t raw -c 1  

Playing raw data 'wbfm-mono.raw' : Signed 16 bit Little Endian, Rate 45600 Hz, Mono
